# WeatherPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [5]:
import requests
import json
import matplotlib.pyplot as plt
import pandas as pd
from citipy import citipy
# import citipy
import numpy as np
from config import api_key
from pprint import pprint

## Generate Cities List

In [6]:
# Get random numbers for latitude and longitude then place in a dataframe

city_df=pd.DataFrame({"long":(np.random.uniform(-180,180,15)), "lat":(np.random.uniform(-90,90,15))})
# city_df.head()
# city_df.tail()
city_df

,long,lat
0,-1.756450,-34.608911
1,-125.218844,-3.241424
2,172.513050,-7.942806
3,-46.667703,33.084390
4,-129.379503,-20.672883
5,122.963980,-39.268951
6,-105.637219,27.344176
7,178.399123,59.237646
8,-155.937631,-33.526197
9,-67.095838,-21.086051


# Use Citipy to Retrieve City Names

In [7]:
# Create lists to store city and country names
# Source: https://github.com/wingchen/citipy
citi=[]
country=[]

for index, row in city_df.iterrows():
    latitude=row['lat']
    longitude=row['long']
    city=citipy.nearest_city(latitude,longitude)
    cityname=city.city_name
    citi.append(cityname)
    cntryname=city.country_code
    country.append(cntryname)
print(citi)
print(country)

# Adds new lists to dataframe
city_df['City']=citi
city_df['Country']=country
city_df

['luderitz', 'atuona', 'lolua', 'torbay', 'rikitea', 'esperance', 'valle de allende', 'beringovskiy', 'avera', 'uyuni', 'nushki', 'mataura', 'busselton', 'cape town', 'albany']
['na', 'pf', 'tv', 'ca', 'pf', 'au', 'mx', 'ru', 'pf', 'bo', 'pk', 'pf', 'au', 'za', 'au']


,long,lat,City,Country
0,-1.756450,-34.608911,luderitz,na
1,-125.218844,-3.241424,atuona,pf
2,172.513050,-7.942806,lolua,tv
3,-46.667703,33.084390,torbay,ca
4,-129.379503,-20.672883,rikitea,pf
5,122.963980,-39.268951,esperance,au
6,-105.637219,27.344176,valle de allende,mx
7,178.399123,59.237646,beringovskiy,ru
8,-155.937631,-33.526197,avera,pf
9,-67.095838,-21.086051,uyuni,bo


### Perform API Calls
* Perform a weather check on each city using a series of successive API calls.
* Include a print log of each city as it'sbeing processed (with the city number and city name).


In [19]:
# https://api.openweathermap.org/data/2.5/weather?q=London,uk&appid=api_key&units=imperial
# https://api.openweathermap.org/data/2.5/weather?&units=imperial&appid=api_key&q=London,uk

url = "http://api.openweathermap.org/data/2.5/weather?"
units = 'imperial'
query_url=f"{url}&units={units}&appid={api_key}&q="
# query_url = f"{url}appid={api_key}&units={units}&q="

cityid = []
long = []
lat = []
temp = []
humidity = []
clouds = []
wind = []
maxtemp = []
date = []
cityagain=[]
countryagain=[]
# # Build query URL
# check if data is available before appending
# for City in city_df:
#     response=requests.get(query_url + cityposition +","+ cntryposition).json()
    
#     try:
#         if response["message"]=="city not found":
#             print("The city can't be found")
#     except:
#         cityagain.append(City)
#         countryagain.append(response['sys']['country'])
#         cityid.append(response['id'])
#         long.append(response['coord']['lon'])
#         lat.append(response['coord']['lat'])
#         temp.append(response['main']['temp'])
#         humidity.append(response['main']['humidity'])
#         clouds.append(response['clouds']['all'])
#         wind.append(response['wind']['speed'])
#         maxtemp.append(response['main']['temp_max'])
#         date.append(response['dt'])
#         print(f'The city name is: {city}')
#         print(f'The city ID is: {response["id"]}')
#         print(f'The longitude information received is: {response["coord"]["lon"]}')
#         print(f'The latitude information received is: {response["coord"]["lat"]}')
#         print(f'The longitude information received is: {response["coord"]["lon"]}')
#         print(f'The temperature is: {response["main"]["temp"]}')
#         print(f'The humidity is: {response["main"]["humidity"]}')
#         print(f'The cloudiness is: {response["clouds"]["all"]}')
#         print(f'The wind speed is: {response["wind"]["speed"]}')
#         print(f'The high is: {response["main"]["temp_max"]}')
#         print(f"                          ")
        
for index, row in city_df.iterrows():
    cityposition=row['City']
    cntryposition= row['Country']
    response=requests.get(query_url + cityposition +","+ cntryposition).json()
    # pprint(response)
    
    try:
        if response["message"]=="city not found":
            print(f"The city can't be found")
            print(f"                          ")
            pass
                  
    except:
        if len(cityid)<501:
            cityagain.append(response['name'])
            countryagain.append(response['sys']['country'])
            cityid.append(response['id'])
            long.append(response['coord']['lon'])
            lat.append(response['coord']['lat'])
            temp.append(response['main']['temp'])
            humidity.append(response['main']['humidity'])
            clouds.append(response['clouds']['all'])
            wind.append(response['wind']['speed'])
            maxtemp.append(response['main']['temp_max'])
            date.append(response['dt'])
            print(f'The city name is: {response["name"]}')
            print(f'The country is: {response["sys"]["country"]}')
            print(f'The city ID is: {response["id"]}')
            print(f'The longitude information received is: {response["coord"]["lon"]}')
            print(f'The latitude information received is: {response["coord"]["lat"]}')
            print(f'The longitude information received is: {response["coord"]["lon"]}')
            print(f'The temperature is: {response["main"]["temp"]}')
            print(f'The humidity is: {response["main"]["humidity"]}')
            print(f'The cloudiness is: {response["clouds"]["all"]}')
            print(f'The wind speed is: {response["wind"]["speed"]}')
            print(f'The high is: {response["main"]["temp_max"]}')
            print(f"                          ")
        else:
            break

The city name is: Lüderitz
The country is: NA
The city ID is: 3355672
The longitude information received is: 15.16
The latitude information received is: -26.65
The longitude information received is: 15.16
The temperature is: 61.48
The humidity is: 39
The cloudiness is: 0
The wind speed is: 16.51
The high is: 61.48
                          
The city name is: Atuona
The country is: PF
The city ID is: 4020109
The longitude information received is: -139.03
The latitude information received is: -9.8
The longitude information received is: -139.03
The temperature is: 78.62
The humidity is: 71
The cloudiness is: 82
The wind speed is: 14.12
The high is: 78.62
                          
The city can't be found
                          
The city name is: Torbay
The country is: CA
The city ID is: 6167817
The longitude information received is: -52.73
The latitude information received is: 47.67
The longitude information received is: -52.73
The temperature is: 39.47
The humidity is: 86
The cloudine

### Convert Raw Data to DataFrame
* Export the city data into a .csv.
* Display the DataFrame

In [29]:
# Create dictionary of lists above
weather={"City":cityagain,"Country":countryagain,"City ID":cityid,"Latitude":lat,"Longitude":long,"Temperature":temp,"Max Temp":maxtemp,"Humidity":humidity,"Wind":wind,"Clouds":clouds,"Date":date}

# Make a dataframe from the dictionary
weather_df=pd.DataFrame(weather)
weather_df

# Save the updated dataframe as a csv
weather_df.to_csv("../output_data/weather_file.csv", encoding="utf-8", index=False)

## Inspect the data and remove the cities where the humidity > 100%.
----
Skip this step if there are no cities that have humidity > 100%. 

In [ ]:
#  Get the indices of cities that have humidity over 100%.


In [ ]:
# Make a new DataFrame equal to the city data to drop all humidity outliers by index.
# Passing "inplace=False" will make a copy of the city_data DataFrame, which we call "clean_city_data".


## Plotting the Data
* Use proper labeling of the plots using plot titles (including date of analysis) and axes labels.
* Save the plotted figures as .pngs.

## Latitude vs. Temperature Plot

## Latitude vs. Humidity Plot

## Latitude vs. Cloudiness Plot

## Latitude vs. Wind Speed Plot

## Linear Regression

####  Northern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Southern Hemisphere - Max Temp vs. Latitude Linear Regression

####  Northern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Humidity (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Southern Hemisphere - Cloudiness (%) vs. Latitude Linear Regression

####  Northern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

####  Southern Hemisphere - Wind Speed (mph) vs. Latitude Linear Regression

In [10]:
 Save config information
url = "http://api.openweathermap.org/data/2.5/weather?"
city = "Chicago"

# Build query URL
query_url = url + "appid=" + api_key + "&q=" + city

# Get weather data
weather_response = requests.get(query_url)
weather_json = weather_response.json()

# Get the temperature from the response
print(f"The weather API responded with: {weather_json}.")
pprint(weather_json)

SyntaxError: invalid syntax (<ipython-input-10-5950657c1d21>, line 1)

In [ ]:
# print(f"The ID of the city is: {cityid}")
# print(f"The latitude information received is: {lat}")
# print(f"The longitude information received is: {long}")
# print(f"The temperature information received is: {temp}")
# print(f"The humidity information received is: {humidity}")
# print(f"The cloud information received is: {clouds}")
# print(f"The wind speed information received is: {wind}")
# print(f"The maximum temperature received is: {maxtemp}")
# print(f"The date received is: {date}")
# print(f"                                                ")
    
    
# url = "http://api.openweathermap.org/data/2.5/weather?"
# units = "imperial"

# # # Build partial query URL
# query_url = f"{url}appid={api_key}&units={units}&q="

#     country=city.country_code
#     country.append(country)
# print(citi)
# print(country)
# cities = ["Paris", "London", "Oslo", "Beijing"]

# url="http://api.openweathermap.org/data/2.5/weather?lat={87.6298}&lon={41.8781}&appid={api_key}"
# # url="http://api.openweathermap.org/data/2.5/weather?q={Chicago}&appid={api_key}?"
# citydata = requests.get(url).json()
# print(json.dumps(citydata, indent=4, sort_keys=True))

# Print the json
# print(geo_data)
# response=requests.get(f"{query_url}'lat='{87.6298}'&lon='{41.8781}").json()
# print(response)

# # set up lists to hold reponse info
# citi = []
# cityid = []
# long = []
# lat = []
# temp = []
# humidity = []
# clouds = []
# wind = []
# country = []

# for index, row in city_df.iterrows():
#     latitude=row['lat']
#     longitude=row['long']
#     response=requests.get(f"{query_url}'lat='{latitude}'&lon='{longitude}").json()
#     print(response)
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])

# Loop through the list of cities and perform a request for data on each
# api.openweathermap.org/data/2.5/find?q=London&units=imperial
# for city in cities:
    
#     response = requests.get(query_url + city).json()
#     citi.append(response['name'])
#     cityid.append(response['sys']['id'])
#     long.append(response['coord']['lon'])
#     lat.append(response['coord']['lat'])
#     temp.append(response['main']['temp'])
#     humidity.append(response['main']['humidity'])
#     clouds.append(response['clouds']['all'])
#     wind.append(response['wind']['speed'])
#     country.append(response['sys']['country'])
    
# print(f"The ID of the city is: {cityid}")
# print(f"The city information received is: {citi}")
# print(f"The country information received is: {country}")
# print(f"The latitude information received is: {lat}")
# print(f"The longitude information received is: {long}")
# print(f"The temperature information received is: {temp}")
# print(f"The humidity information received is: {humidity}")
# print(f"The cloud information received is: {clouds}")
# print(f"The wind speed information received is: {wind}")



# # Save config information
# url = "http://api.openweathermap.org/data/2.5/weather?"
# city = "Chicago"

# # Build query URL
# query_url = url + "appid=" + api_key + "&q=" + city

# # Get weather data
# weather_response = requests.get(query_url)
# weather_json = weather_response.json()

# # Get the temperature from the response
# print(f"The weather API responded with: {weather_json}.")
# pprint(weather_json)